In [1]:
%matplotlib inline



# Model Selection




In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import itertools
import pyproj
import verde as vd

data = vd.datasets.fetch_texas_wind()

# Use Mercator projection because Spline is a Cartesian gridder
projection = pyproj.Proj(proj="merc", lat_ts=data.latitude.mean())
proj_coords = projection(data.longitude.values, data.latitude.values)

region = vd.get_region((data.longitude, data.latitude))
# The desired grid spacing in degrees (converted to meters using 1 degree approx. 111km)
spacing = 15 / 60

Before we begin tuning, let's reiterate what the results were with the default
parameters.



In [4]:
spline_default = vd.Spline()
score_default = np.mean(
    vd.cross_val_score(spline_default, proj_coords, data.air_temperature_c)
)
spline_default.fit(proj_coords, data.air_temperature_c)
print("R² with defaults:", score_default)

R² with defaults: 0.796036885482746


## Tuning

:class:`~verde.Spline` has many parameters that can be set to modify the final result.
Mainly the ``damping`` regularization parameter and the ``mindist`` "fudge factor"
which smooths the solution.



In [5]:
dampings = [None, 1e-4, 1e-3, 1e-2]
mindists = [5e3, 10e3, 50e3, 100e3]

# Use itertools to create a list with all combinations of parameters to test
parameter_sets = [
    dict(damping=combo[0], mindist=combo[1])
    for combo in itertools.product(dampings, mindists)
]
print("Number of combinations:", len(parameter_sets))
print("Combinations:", parameter_sets)

Number of combinations: 16
Combinations: [{'damping': None, 'mindist': 5000.0}, {'damping': None, 'mindist': 10000.0}, {'damping': None, 'mindist': 50000.0}, {'damping': None, 'mindist': 100000.0}, {'damping': 0.0001, 'mindist': 5000.0}, {'damping': 0.0001, 'mindist': 10000.0}, {'damping': 0.0001, 'mindist': 50000.0}, {'damping': 0.0001, 'mindist': 100000.0}, {'damping': 0.001, 'mindist': 5000.0}, {'damping': 0.001, 'mindist': 10000.0}, {'damping': 0.001, 'mindist': 50000.0}, {'damping': 0.001, 'mindist': 100000.0}, {'damping': 0.01, 'mindist': 5000.0}, {'damping': 0.01, 'mindist': 10000.0}, {'damping': 0.01, 'mindist': 50000.0}, {'damping': 0.01, 'mindist': 100000.0}]


Now we can loop over the combinations and collect the scores for each parameter set.



In [6]:
spline = vd.Spline()
scores = []
for params in parameter_sets:
    spline.set_params(**params)
    score = np.mean(vd.cross_val_score(spline, proj_coords, data.air_temperature_c))
    scores.append(score)
print(scores)

[-6.672752082907465, 0.48454031729297853, 0.8383522700289824, 0.8371988991061127, 0.8351153077361344, 0.8316607509398537, 0.849262993054378, 0.8418400888039852, 0.8371795091867389, 0.8412200336704057, 0.8529555082418581, 0.852172766760899, 0.8401945161936075, 0.8330182923874666, 0.844170645856404, 0.8491145591355839]


The largest score will yield the best parameter combination.



In [7]:
best = np.argmax(scores)
print("Best score:", scores[best])
print("Score with defaults:", score_default)
print("Best parameters:", parameter_sets[best])

Best score: 0.8529555082418581
Score with defaults: 0.796036885482746
Best parameters: {'damping': 0.001, 'mindist': 50000.0}


## Cross-validated gridders

The :class:`verde.SplineCV` class provides a cross-validated version of
:class:`verde.Spline`. It has almost the same interface but does all of the above
automatically when fitting a dataset. The only difference is that you must provide a
list of ``damping`` and ``mindist`` parameters to try instead of only a single value:



In [8]:
spline = vd.SplineCV(
    dampings=dampings,
    mindists=mindists,
)

Calling :meth:`~verde.SplineCV.fit` will run a grid search over all parameter
combinations to find the one that maximizes the cross-validation score.



In [9]:
spline.fit(proj_coords, data.air_temperature_c)

SplineCV(dampings=[None, 0.0001, 0.001, 0.01],
         mindists=[5000.0, 10000.0, 50000.0, 100000.0])

The estimated best damping and mindist, as well as the cross-validation
scores, are stored in class attributes:



In [10]:
print("Highest score:", spline.scores_.max())
print("Best damping:", spline.damping_)
print("Best mindist:", spline.mindist_)

Highest score: 0.8529555082418581
Best damping: 0.001
Best mindist: 50000.0


The cross-validated gridder can be used like any other gridder (including in
:class:`verde.Chain` and :class:`verde.Vector`):



In [11]:
grid = spline.grid(
    region=region,
    spacing=spacing,
    projection=projection,
    dims=["latitude", "longitude"],
    data_names="temperature",
)
print(grid)

<xarray.Dataset>
Dimensions:      (latitude: 43, longitude: 51)
Coordinates:
  * longitude    (longitude) float64 -106.4 -106.1 -105.9 ... -94.06 -93.8
  * latitude     (latitude) float64 25.91 26.16 26.41 ... 35.91 36.16 36.41
Data variables:
    temperature  (latitude, longitude) float64 19.42 19.42 19.43 ... 7.536 7.765
Attributes:
    metadata:  Generated by SplineCV(dampings=[None, 0.0001, 0.001, 0.01],\n ...


Like :func:`verde.cross_val_score`, :class:`~verde.SplineCV` can also run the
grid search in parallel using `Dask <https://dask.org/>`__ by specifying the
``delayed`` attribute:



In [12]:
spline = vd.SplineCV(dampings=dampings, mindists=mindists, delayed=True)

Unlike :func:`verde.cross_val_score`, calling :meth:`~verde.SplineCV.fit`
does **not** result in :func:`dask.delayed` objects. The full grid search is
executed and the optimal parameters are found immediately.



In [13]:
spline.fit(proj_coords, data.air_temperature_c)

print("Best damping:", spline.damping_)
print("Best mindist:", spline.mindist_)

Best damping: 0.001
Best mindist: 50000.0


The one caveat is the that the ``scores_`` attribute will be a list of
:func:`dask.delayed` objects instead because the scores are only computed as
intermediate values in the scheduled computations.



In [14]:
print("Delayed scores:", spline.scores_)

Delayed scores: [Delayed('mean-8c8375a1-12b0-4327-9ab1-74098396f5b6'), Delayed('mean-53245cb6-9ce4-4b16-8c1d-f80432de54de'), Delayed('mean-948c31ac-4737-4258-9c5f-40db4e2c6f74'), Delayed('mean-73a61014-fd6f-4e2e-88b4-438b13a5fbd8'), Delayed('mean-d53a27ab-2c19-48ff-8911-b66fb7d8180c'), Delayed('mean-7a1fdbc6-68e2-4f39-b4ff-4c3b8419ef68'), Delayed('mean-fbbfd5ef-9d4c-48b9-ae7d-8dec495bc1b7'), Delayed('mean-282126e0-ef8b-4ceb-8e8d-394e6f1d9e24'), Delayed('mean-a482da66-8448-423b-bd31-d6f63e451c3b'), Delayed('mean-8a28470d-5e28-49bf-8c4b-215f985a3fd1'), Delayed('mean-50bb6bff-7c81-42b0-9640-c935f72ef3b2'), Delayed('mean-baba8b0f-3387-4e05-852b-1ea4fe8332fc'), Delayed('mean-6d4c2f91-83b2-4218-861d-3f726ea5cec0'), Delayed('mean-25dc3b25-3d67-4512-87f6-361cfa1a646f'), Delayed('mean-9da90c50-3d05-4cb1-86c1-cce47e07eaa4'), Delayed('mean-de6db905-c21e-4912-a5a9-3abcdd9cbbb5')]


Calling :func:`dask.compute` on the scores will calculate their values but
will unfortunately run the entire grid search again. So using
``delayed=True`` is not recommended if you need the scores of each parameter
combination.

